In [ ]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - SMBH Binary SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 31 January 2022
# Last Revision: 31 December 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [23]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol#.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [31]:
# Tsalmantza+2011 report the detection of 9 SMBH binary candidates
# Methodology: They search for systems exhibiting two sets of emission lines in the optical (using SDSS):
#    one broad and one narrow set of emission lines which are kinematically offset. A small separation would be 
#    expected in the case of Keplerian roation due to a binary. Here the narrow lines trace the galaxy, the broad
#    lines trace the active BH.
# Probes redshift 0.1-1.5, and applied fitting technique to SDSS QSO spectra as well as objects classified as galaxies
# All objects selected based on interesting features are at z<0.8
# There seems to be some overlap with Shen+2010 ****

# Loading in objects examined by Tsalmantza
tsalmantza = pd.read_csv('Tsalmantza2011/table1.csv', sep=',') #((Table.read('Tsalmantza2011/table1.dat', format='ascii')).to_pandas())#.drop(columns=['---'])

# Here, we're keeping only objects flagged as candidate binaries
tsalmantza = tsalmantza[(tsalmantza['Class']=='B') | (tsalmantza['Class']=='B;D') | (tsalmantza['Class']=='E;B') | (tsalmantza['Class']=='A;B') | (tsalmantza['Class']=='D;B') | (tsalmantza['Class']=='O;B')].reset_index(drop=True)

tsalmantza['Name1'] = tsalmantza['Name']
tsalmantza['Name2'] = tsalmantza['Name']
tsalmantza['z1'] = tsalmantza['z_NL']
tsalmantza['z2'] = tsalmantza['z_NL']
tsalmantza['z1_type'] = "spec"
tsalmantza['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(tsalmantza,tsalmantza['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = tsalmantza['RA'], dec = tsalmantza['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
tsalmantza['RA_deg'] = coordconvert.ra.degree
tsalmantza['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
tsalmantza['RA2'] = tsalmantza['RA']
tsalmantza['Dec2'] = tsalmantza['Dec']

tsalmantza['RA2_deg'] = tsalmantza['RA_deg']
tsalmantza['Dec2_deg'] = tsalmantza['Dec_deg']

# Adding details about the coordinates
tsalmantza['Equinox'] = "J2000"
tsalmantza['Coordinate_waveband'] = "Optical"
tsalmantza['Coordinate_Source'] = "SDSS"

tsalmantza['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
tsalmantza['Brightness1'] = -100
tsalmantza['Brightness_band1'] = -100
tsalmantza['Brightness_type1'] = -100

tsalmantza['Brightness2'] = -100
tsalmantza['Brightness_band2'] = -100
tsalmantza['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
tsalmantza['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#tsalmantza['Sep(kpc)'] = tsalmantza['Sep']*((cosmo.arcsec_per_kpc_proper(tsalmantza['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
tsalmantza['delta_z'] = tsalmantza['z1']-tsalmantza['z2']
tsalmantza['dV'] = (2.99e+5)*((1+tsalmantza['z1'])**2 - (1+tsalmantza['z2'])**2)/((1+tsalmantza['z1'])**2+(1+tsalmantza['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
tsalmantza['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
tsalmantza['Confirmation Method'] = "-99"
tsalmantza['Paper(s)'] = "Tsalmantza+2011"
tsalmantza['BibCode(s)'] = "2011ApJ...738...20T"
tsalmantza['DOI(s)'] = "https://doi:10.1088/0004-637X/738/1/20"

## And dropping any columns that we don't need....
#tsalmantza.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#tsalmantza = tsalmantza[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#tsalmantza.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

tsalmantza




# Eracleous+2012 report the detection of 
# Redshift range of z<0.7. 
# Trying to probe binaries with separations of subparsecs and periods of a few centuries
# They systematically search SDSS for quasars whose broad (single-peaked) B-beta lines are offset from the expected
# wavelength by 1000 km/s or more, considering these as candidate SMBH binaries or recoiling SMBHs
#     They throw out systems where delta v < 1000 km/s, and systems showing clear DPE signatures.
#     But they retain flat top line profiles as well as line profiles that are displayed but have an extended wing or shoulder
# ---> They stress this is not a complete sample, and is very subjective given that visual classification was used, 
#      and they added two objects from Strateva+03
# ----> There is overlap with Tsalmantza+ and others

# Bonning+200 attempted to find binaries or recoils ...



,Name,RA,Dec,MJD,Plate,Fiber,z_NL,z_BL,Method,Class,Name1,Name2,z1,z2,z1_type,z2_type,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J0927+2943,09:27:12.65,+29:43:44.1,53389,1939,467,0.713,0.698,QG;QQ,B,J0927+2943,J0927+2943,0.713,0.713,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
1,J0932+0318,09:32:01.60,+03:18:58.7,52254,568,39,0.420,0.401,QQ,B;D,J0932+0318,J0932+0318,0.420,0.420,spec,spec,143.006667,3.316306,09:32:01.60,+03:18:58.7,143.006667,3.316306,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
2,J1000+2233,10:00:21.80,+22:33:18.6,53737,2298,102,0.419,0.377,GQ,E;B,J1000+2233,J1000+2233,0.419,0.419,spec,spec,150.090833,22.555167,10:00:21.80,+22:33:18.6,150.090833,22.555167,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
3,J1012+2613,10:12:26.86,+26:13:27.3,53757,2347,513,0.378,0.351,QQ,E;B,J1012+2613,J1012+2613,0.378,0.378,spec,spec,153.111917,26.224250,10:12:26.86,+26:13:27.3,153.111917,26.224250,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
4,J1050+3456,10:50:41.36,+34:56:31.4,53431,2025,603,0.272,0.258,QQ;GQ,B,J1050+3456,J1050+3456,0.272,0.272,spec,spec,162.672333,34.942056,10:50:41.36,+34:56:31.4,162.672333,34.942056,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
5,J1154+0134,11:54:49.42,+01:34:43.6,52051,515,99,0.469,0.450,QQ,A;B,J1154+0134,J1154+0134,0.469,0.469,spec,spec,178.705917,1.578778,11:54:49.42,+01:34:43.6,178.705917,1.578778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
6,J1536+0441,15:36:36.22,+04:41:27.0,54567,1836,270,0.389,0.373,QG;QQ,D;B,J1536+0441,J1536+0441,0.389,0.389,spec,spec,234.150917,4.690833,15:36:36.22,+04:41:27.0,234.150917,4.690833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
7,J1539+3333,15:39:08.09,+33:33:28.0,52823,1355,572,0.226,0.199,QG;QQ,O;B,J1539+3333,J1539+3333,0.226,0.226,spec,spec,234.783708,33.557778,15:39:08.09,+33:33:28.0,234.783708,33.557778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
8,J1714+3327,17:14:48.51,+33:27:38.3,54591,2973,190,0.181,0.186,QQ,O;B,J1714+3327,J1714+3327,0.181,0.181,spec,spec,258.702125,33.460639,17:14:48.51,+33:27:38.3,258.702125,33.460639,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20


,Name,RA,Dec,MJD,Plate,Fiber,z_NL,z_BL,Method,Class
0,J0927+2943,09:27:12.65,+29:43:44.1,53389,1939,467,0.713,0.698,QG;QQ,B
1,J0932+0318,09:32:01.60,+03:18:58.7,52254,568,39,0.420,0.401,QQ,B;D
2,J1000+2233,10:00:21.80,+22:33:18.6,53737,2298,102,0.419,0.377,GQ,E;B
3,J1012+2613,10:12:26.86,+26:13:27.3,53757,2347,513,0.378,0.351,QQ,E;B
4,J1050+3456,10:50:41.36,+34:56:31.4,53431,2025,603,0.272,0.258,QQ;GQ,B
5,J1154+0134,11:54:49.42,+01:34:43.6,52051,515,99,0.469,0.450,QQ,A;B
6,J1536+0441,15:36:36.22,+04:41:27.0,54567,1836,270,0.389,0.373,QG;QQ,D;B
7,J1539+3333,15:39:08.09,+33:33:28.0,52823,1355,572,0.226,0.199,QG;QQ,O;B
8,J1714+3327,17:14:48.51,+33:27:38.3,54591,2973,190,0.181,0.186,QQ,O;B


In [21]:
# Here we're loading in the Shen+2013 catalog and we are ONLY including objects with their binary candidate flag \
# indicated by the (1) flag in  the 'Category' column
# I am not including ambiguous (3) or variability (2) cases
shen2013 = pd.read_csv('Shen2013/table2.csv', sep=',')
shen2013 = shen2013[shen2013['Category']==1.0]

shen2013['Name1'] = shen2013['Designation']
shen2013['Name2'] = shen2013['Designation']
shen2013['z1'] = shen2013['z_sys']
shen2013['z2'] = shen2013['z_sys']
shen2013['z1_type'] = "spec"
shen2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(shen2013,shen2013['Designation'])

shen2013['Coordinates'] = shen2013['Designation']#.str.slice(start=1) # Stripping the J
shen2013['RA_test'] = shen2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
shen2013['Dec_test'] = shen2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
shen2013['RA'] = shen2013['RA_test'].str.slice(start=0, stop=2)+":"+shen2013['RA_test'].str.slice(start=2, stop=4)+":"+shen2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
shen2013['Dec'] = shen2013['Dec_test'].str.slice(start=0, stop=3)+":"+shen2013['Dec_test'].str.slice(start=3, stop=5)+":"+shen2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
shen2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = shen2013['RA'], dec = shen2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
shen2013['RA_deg'] = coordconvert.ra.degree
shen2013['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
shen2013['RA2'] = shen2013['RA']
shen2013['Dec2'] = shen2013['Dec']

shen2013['RA2_deg'] = shen2013['RA_deg']
shen2013['Dec2_deg'] = shen2013['Dec_deg']

# Adding details about the coordinates
shen2013['Equinox'] = "J2000"
shen2013['Coordinate_waveband'] = "Optical"
shen2013['Coordinate_Source'] = "SDSS"

shen2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
shen2013['Brightness1'] = -100
shen2013['Brightness_band1'] = -100
shen2013['Brightness_type1'] = -100

shen2013['Brightness2'] = -100
shen2013['Brightness_band2'] = -100
shen2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
shen2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#shen2013['Sep(kpc)'] = shen2013['Sep']*((cosmo.arcsec_per_kpc_proper(shen2013['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
shen2013['delta_z'] = shen2013['z1']-shen2013['z2']
shen2013['dV'] = (2.99e+5)*((1+shen2013['z1'])**2 - (1+shen2013['z2'])**2)/((1+shen2013['z1'])**2+(1+shen2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
shen2013['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
shen2013['Confirmation Method'] = "-99"
shen2013['Paper(s)'] = "Shen+2013"
shen2013['BibCode(s)'] = "2013ApJ...775...49S"
shen2013['DOI(s)'] = "https://doi:10.1088/0004-637X/775/1/49"

## And dropping any columns that we don't need....
#shen2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#shen2013 = shen2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#shen2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


shen2013

,Designation,z_sys,logM_1(M_sun),R_BLR(pc),E1Plate,E1Fiber,E1MJD,E2Plate,E2Fiber,E2MJD,Deltat(days),$V_{\rm ccf}^*$(km s^-1),$V_{\rm ccf}^*$(km s^-1).1,$V_{\rm ccf}^*$(km s^-1).2,$a_{\rm ccf}^*$(km s^-1 yr^-1),$a_{\rm ccf}^*$(km s^-1 yr^-1).1,$a_{\rm ccf}^*$(km s^-1 yr^-1).2,Category,Name1,Name2,z1,z2,z1_type,z2_type
3,030100.23+000429.3,0.486,8.50,0.056,411,265,51817,709,477,52205,261,420,163,545,588,229.0,762.0,1.0,030100.23+000429.3,030100.23+000429.3,0.486,0.486,spec,spec
7,032213.89+005513.4,0.185,8.04,0.068,414,341,51901,1181,329,53358,1229,48,21,78,14,6.0,23.0,1.0,032213.89+005513.4,032213.89+005513.4,0.185,0.185,spec,spec
13,122909.52-003530.0,0.450,8.84,0.056,289,150,51990,2568,32,54153,1491,324,197,453,79,48.0,111.0,1.0,122909.52-003530.0,122909.52-003530.0,0.450,0.450,spec,spec
18,141020.57+364322.7,0.450,8.44,0.053,1643,359,53143,2931,496,54590,998,227,142,309,83,52.0,113.0,1.0,141020.57+364322.7,141020.57+364322.7,0.450,0.450,spec,spec
19,153705.95+005522.8,0.137,7.63,0.038,2955,289,54562,315,419,51663,2550,110,43,181,15,6.0,26.0,1.0,153705.95+005522.8,153705.95+005522.8,0.137,0.137,spec,spec
21,155053.16+052112.1,0.110,8.96,0.039,1822,308,53172,2951,326,54592,1278,-1034,-1183,-882,-295,-338.0,-251.0,1.0,155053.16+052112.1,155053.16+052112.1,0.110,0.110,spec,spec
28,234932.77-003645.8,0.279,8.26,0.068,386,215,51788,684,261,52523,574,-68,-118,-17,-43,-75.0,-11.0,1.0,234932.77-003645.8,234932.77-003645.8,0.279,0.279,spec,spec


In [18]:
# Here we're loading in the Eracleous+2012 table(s)

erac2012 = ((Table.read('Eracleous2012/table1.dat', readme = 'Eracleous2012/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

# Need to add in other info for the 68 candiates follow-up and the 14 specific candidates they discuss



erac2012.drop_duplicates(subset=['SDSS'], inplace=True)
erac2012.reset_index(drop=True, inplace=True)

erac2012['Name1'] = erac2012['SDSS']
erac2012['Name2'] = erac2012['SDSS']
erac2012['z1'] = erac2012['z']
erac2012['z2'] = erac2012['z']
erac2012['z1_type'] = "spec"
erac2012['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(erac2012,erac2012['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = erac2012['RA'], dec = erac2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
erac2012['RA_deg'] = coordconvert.ra.degree
erac2012['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
erac2012['RA2'] = erac2012['RA']
erac2012['Dec2'] = erac2012['Dec']

erac2012['RA2_deg'] = erac2012['RA_deg']
erac2012['Dec2_deg'] = erac2012['Dec_deg']

# Adding details about the coordinates
erac2012['Equinox'] = "J2000"
erac2012['Coordinate_waveband'] = "Optical"
erac2012['Coordinate_Source'] = "SDSS"

erac2012['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
erac2012['Brightness1'] = erac2012['Vmag']
erac2012['Brightness_band1'] = 'V'
erac2012['Brightness_type1'] = 'apparent mag'

erac2012['Brightness2'] = erac2012['Vmag']
erac2012['Brightness_band2'] = 'V'
erac2012['Brightness_type2'] = 'apparent mag'

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
erac2012['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#erac2012['Sep(kpc)'] = erac2012['Sep']*((cosmo.arcsec_per_kpc_proper(erac2012['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
erac2012['delta_z'] = erac2012['z1']-erac2012['z2']
erac2012['dV'] = (2.99e+5)*((1+erac2012['z1'])**2 - (1+erac2012['z2'])**2)/((1+erac2012['z1'])**2+(1+erac2012['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
erac2012['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
erac2012['Confirmation Method'] = "-99"
erac2012['Paper(s)'] = "Eracleous+2012"
erac2012['BibCode(s)'] = "2012ApJS..201...23E"
erac2012['DOI(s)'] = "doi:10.1088/0067-0049/201/2/23"

## And dropping any columns that we don't need....
#erac2012.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#erac2012 = erac2012[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#erac2012.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


erac2012

,SDSS,z,Vmag,AV,VMag,Date,Instr,Exp,S/N,lam1,lam2,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,001224.02-102226.2,0.2287,17.07,0.127,-23.27,2001/08/20,S,3756,29,3093,7505,001224.02-102226.2,001224.02-102226.2,0.2287,0.2287,spec,spec,00:12:24.02,-10:22:26.2,3.100083,-10.373944,00:12:24.02,-10:22:26.2,3.100083,-10.373944,J2000,Optical,SDSS,Binary AGN Candidate,17.07,V,apparent mag,17.07,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
1,002444.10+003221.3,0.4024,16.85,0.083,-24.86,2000/12/22,S,9900,88,2709,6594,002444.10+003221.3,002444.10+003221.3,0.4024,0.4024,spec,spec,00:24:44.10,+00:32:21.3,6.183750,0.539250,00:24:44.10,+00:32:21.3,6.183750,0.539250,J2000,Optical,SDSS,Binary AGN Candidate,16.85,V,apparent mag,16.85,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
2,015530.01-085704.0,0.1648,16.84,0.080,-22.65,2001/09/16,S,2702,32,3263,7917,015530.01-085704.0,015530.01-085704.0,0.1648,0.1648,spec,spec,01:55:30.01,-08:57:04.0,28.875042,-8.951111,01:55:30.01,-08:57:04.0,28.875042,-8.951111,J2000,Optical,SDSS,Binary AGN Candidate,16.84,V,apparent mag,16.84,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
3,020011.53-093126.2,0.3602,17.96,0.081,-23.46,2001/08/28,S,8104,34,2795,6780,020011.53-093126.2,020011.53-093126.2,0.3602,0.3602,spec,spec,02:00:11.53,-09:31:26.2,30.048042,-9.523944,02:00:11.53,-09:31:26.2,30.048042,-9.523944,J2000,Optical,SDSS,Binary AGN Candidate,17.96,V,apparent mag,17.96,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
4,021259.59-003029.5,0.3945,17.87,0.117,-23.83,2000/09/29,S,2700,29,2714,6613,021259.59-003029.5,021259.59-003029.5,0.3945,0.3945,spec,spec,02:12:59.59,-00:30:29.5,33.248292,-0.508194,02:12:59.59,-00:30:29.5,33.248292,-0.508194,J2000,Optical,SDSS,Binary AGN Candidate,17.87,V,apparent mag,17.87,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
5,022014.57-072859.1,0.2137,18.65,0.078,-21.47,2001/09/10,S,4501,11,3133,7598,022014.57-072859.1,022014.57-072859.1,0.2137,0.2137,spec,spec,02:20:14.57,-07:28:59.1,35.060708,-7.483083,02:20:14.57,-07:28:59.1,35.060708,-7.483083,J2000,Optical,SDSS,Binary AGN Candidate,18.65,V,apparent mag,18.65,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
6,031715.87+425150.6,0.2356,19.03,0.616,-21.87,2003/12/03,S,13001,17,3075,7463,031715.87+425150.6,031715.87+425150.6,0.2356,0.2356,spec,spec,03:17:15.87,+42:51:50.6,49.316125,42.864056,03:17:15.87,+42:51:50.6,49.316125,42.864056,J2000,Optical,SDSS,Binary AGN Candidate,19.03,V,apparent mag,19.03,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
7,074007.28+410903.6,0.2347,17.78,0.161,-22.66,2000/12/06,S,5400,23,3074,7468,074007.28+410903.6,074007.28+410903.6,0.2347,0.2347,spec,spec,07:40:07.28,+41:09:03.6,115.030333,41.151000,07:40:07.28,+41:09:03.6,115.030333,41.151000,J2000,Optical,SDSS,Binary AGN Candidate,17.78,V,apparent mag,17.78,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
8,074157.25+275519.8,0.3252,18.52,0.170,-22.73,2002/03/06,S,4802,19,2867,6958,074157.25+275519.8,074157.25+275519.8,0.32

In [ ]:
# Here we're loading in the Ju+2013 table(s)





In [ ]:
# Here we're loading in the Decarli+2013 table, which we'll need to match on the Tsalmantza+2011 table since \
# the sample in Decarli+2013 are pulled directly from the binary candidates in Tsalmantza+2011.

                
                         

In [ ]:
###

# HENNAWI 2013 IS PROJECTED QUASAR PAIRS WITH VELOCITY DIFFERENCES >2000 KM S^-1, NO NEED TO INCLUDE.
# Prochaska+2013 are also projected pairs with >2000 km s^-1
# Lau+2018 and Lau+2016 required velocity differences >3000 km s^-1
# Rubni 2015 expunged all systems within 5000 km s^-1 of the foregrouns quasar